In [1]:
import pandas as pd
import io, gzip
from codecs import decode
import pysrt
import datetime
import base64
import re
from tqdm import tqdm
from os import listdir
import pdb
import dill
import time
import numpy as np
import sys
from xmlrpc.client import ServerProxy
import os as os
import os.path
import json


In [2]:
with open('credentials.pkd','rb') as fp:
    credentials = dill.load(fp)
USER_AGENT=credentials['user']
USERNAME=credentials['user']
PASS=credentials['passw']

In [3]:
OPENSUBTITLES_SERVER = 'http://api.opensubtitles.org/xml-rpc'
LANGUAGE = 'en'

xmlrpc = ServerProxy(OPENSUBTITLES_SERVER,allow_none=True)
data = xmlrpc.LogIn(USERNAME, PASS, LANGUAGE, USER_AGENT)
status=data.get('status')
if status != '200 OK':
    print('Could not log in. Something is wrong... ')
    print("data.get('status')")
    token = None
else:
    token = data.get('token')
    print('Logged in. Acquired token = {}'.format(token))
def server_stat():
    print('------------------Current Stats-----------------')
    for key,item in xmlrpc.ServerInfo(token)['download_limits'].items():
        print('{}: {}'.format(key,item))
    return
def get_down_quota():
    return xmlrpc.ServerInfo(token)['download_limits']['client_download_quota']

Logged in. Acquired token = Ioi9AY-QcMZr8agB5dFxai50yB2


In [4]:
server_stat()

------------------Current Stats-----------------
global_24h_download_limit: 200
client_ip: 216.174.124.203
limit_check_by: user_id
user_id: 5937493
client_24h_download_count: 1
client_download_quota: 999
client_24h_download_limit: 1000


In [5]:
data_path=os.getcwd()+'\\website\\flask_files\\model_db\\'
subs_dir=os.getcwd()+'\\subs_storage\\'
import pickle
def save_obj(obj, name):
    with open(data_path + name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, protocol=2)
        
def load_obj(name):
    try:
        with open(data_path + name + '.pkd', 'rb+') as fp:
            return pickle.load(fp)
    except:
        with open(data_path + name + '.pkl', 'rb+') as fp:
            return pickle.load(fp)

In [6]:
movie_database=[]
for item in [item.split(".")[0] for item in os.listdir(data_path) if "movie_DB" in item  and "temp" not in item]:
    movie_database+=load_obj(item)

In [7]:
def mdb_unique(movie_db):
    return dict([(item['imdb_id'],item) for item in movie_database])

In [8]:
mdbu=mdb_unique(movie_database)
print('number movies in db:{}'.format(len(mdbu)))

number movies in db:50366


# Step 1 - Subtitle_ID's
Using out database of IMDB_ID's we seek 3 Subtitle_ID's in order of quality. Best one is first. We will then attempt download of the best, and if that doesnt work, the next one and so on... We do the search in batches of 500 whichis the maximum allowed.


In [9]:
def filter_one(sr_one):
    if not sr_one:
        return ()
    else:
        try:
            sr=[(item['SubDownloadsCnt'],item['SubRating'],item['IDSubtitleFile'],item['IDMovieImdb']) for item in sr_one 
             if item['SubFormat']=='srt' if not re.findall('.*[^a-zA-Z]cd[^a-zA-Z]',item['MovieReleaseName'].lower())]
            sr_sort=sorted(sr, key=lambda x: -int(x[0]))
            result=tuple([fill_id(sr_sort[0][3])]+[item[2] for item in sr_sort[0:3]])
            #pdb.set_trace()
            return result #returns a tuple 
        except:
            return ()

def fill_id(string):
    #opensubtitles removes the 'tt' and all leading 0's . this function replaces them.
    while len(string) < 7:
        string = '0' + string
    return 'tt' + string
    
def search_imdbid(list_imdbids):
    #SearchSubtitles is crappy because it also gives you 500 search results
    #useless to search for 500 movies at a time since each one might have ~100 results
    #for safety i will search for 10 at a time ~ 2000 requests ~ 40per 10seconds ~ 10-15 mins maybe 
    search_params=[{'sublanguageid':'eng','imdbid':item.strip('tt')} for item in list_imdbids]
    search_data=xmlrpc.SearchSubtitles(token,search_params)
    sr=[]
    try_num=0
    while sr==[]:
        if search_data['status'] == '200 OK':
            sd=search_data['data'] #a long list of search results by imdb_ids
            uniq_id=list(set([item['IDMovieImdb'] for item in sd]))
            sr=[filter_one([item for item in sd if item['IDMovieImdb'] == idnum]) for idnum in uniq_id] 
            #filter_one takes a list of search results for a single imdbid and returns a tuple (imdbid, subid1,subid2,subid3)
            #the above 3 lines will partition the search results sd into a list of lists where each inner list is for one imdbid
            if sr==[]: #sr will be an empty list if no results are found (it is possible)
                return []         
        else:
            if try_num==3: #gives up after 3 times
                print('Tried 3 times. Giving up.')
                return 
            try_num=try_num+1
            print('Error Code: {} Retrying in 10 seconds.'.format(search_data['status']))    
            time.sleep(10)
    return sr

# Take movie_database-queries opebsubs-returns sub_id list 

In [ ]:
t0 = time.time()
sub_ids=[]
LA=list(mdbu.keys())
size=10
for i in tqdm(range(0,10000,size)):
    sub_ids=sub_ids+search_imdbid(LA[i:i+size])
    if i%2000==0:
        #print('\nWriting data to file. {} movies so far. {:.3f} Percent Complete.'.format(i,(i/N_movies)*100))
        save_obj(sub_ids, datetime.datetime.now().strftime('sub_id_%Y_%b_%d_final'))
        
sub_id_clean=[item for item in sub_ids if item!=()]
save_obj(sub_id_clean, datetime.datetime.now().strftime('sub_id_%Y_%b_%d_final'))

print('\nElapsed {} seconds.'.format(time.time()-t0))    

 72%|████████████████████████████████████████████████████████████████▍                        | 724/1000 [55:06<21:00,  4.57s/it]

In [7]:
sub_id_clean = load_obj('sub_id_clean')
print('Number of sub_ids {}'.format(len(sub_id_clean)))

Number of sub_ids 16893


# Misc Functions

In [38]:
def get_title(imdb_id):
    try:
        ind=[item['imdb_id'] for item in movie_database].index(imdb_id)
        return movie_database[ind]['title'] 
    except:
        return 'Not Found.'

# Downloading the subs

In [39]:
def down_sub(sub_id):
    downsub=xmlrpc.DownloadSubtitles(token, [sub_id])
    if downsub['status'] != '200 OK':
        print('Error Code: {}'.format(downsub['status']))
        server_stat()
        sys.exit('Leaving this world now.')
    return downsub

def decode_sub(enc_subfile):
    dec_data=decode(bytearray(enc_subfile['data'][0]['data'].encode("utf-8")),'base64')
    try:
        sub_txt=gzip.GzipFile(fileobj=io.BytesIO(dec_data)).read().decode("utf-8")       
    except:
        sub_txt=gzip.GzipFile(fileobj=io.BytesIO(dec_data)).read().decode("ISO-8859-1")   
    return sub_txt

def write_to_file(file_name, file_content):
    #writes file_content into an srt file in the subs_storage subfolder of current path
    #if exists prints message
    #if os.path.isfile(os.getcwd()+'\\subs_storage\\'+ file_name + '.srt'):
    #    print('Sub '+ file_name + ' exists already.')
    #else:
    with open(os.getcwd()+'\\subs_storage\\'+ file_name +'.srt','wb') as fp:
        fp.write(file_content.encode('utf-8'))

# THIS PART ACTIVELY DOWNLOADS
Checks to see what *.srt files exist in the folder. 
1. If they are there nothing happens.
2. If they are not there then it uses sub_id_clean file to get the best sub listing. 

<img src="http://clipartix.com/wp-content/uploads/2017/06/Stop-sign-clip-art-microsoft-free-clipart-images-2.png" alt="Le bag of words" title="The bag of words" />

**RUNNING THE NEXT CELL WILL CAUSE DOWNLOADING TO OCCUR. YOU ONLY HAVE 1000 PER DAY.**
**MAKE SURE EVERYTHING IS GOOD BEFORE RUNINNG**

In [13]:
for item in sub_id_clean:
    if os.path.isfile(os.getcwd()+'\\subs_storage\\'+ item[0] + '.srt'):
        continue
        #print('SUB_ID:{:<10} IMDB_ID:{} exists ---- SKIPPING FILE Title: {}'.format(item[1],item[0],get_title(item[0])))
    else:
        print('SUB_ID:{:<10} IMDB_ID:{} downloading ------------- Title: {}'.format(item[1],item[0],get_title(item[0])))
        write_to_file(item[0],decode_sub(down_sub(item[1])))

# Parsing SRT Files

In [135]:
subs_dir=os.getcwd()+'\\subs_storage\\'
def read_srt(imdb_id):
    if os.path.isfile(subs_dir+imdb_id + '.srt'):
        with open(subs_dir+imdb_id+'.srt','rb') as fp:
            test=fp.read().decode('utf-8')
        return test
    else:
        pass

def fparse1(string):
    if string:
        #string=re.sub(r'<i>','',string)    
        #string=re.sub(r'</i>','',string)
        string=re.sub('<[^>]*.','',string) #kills anything inside < CACA >
        string=re.sub(r'\ufeff','',string)
        #string=re.sub('^[^(1)]*','',string) #removes stuff before the 1
        string=re.sub('^[^\a]*?1\r\n00:','1\r\n00:',string) #removes stuff before the 1 betterly
        return string
    else:
        pass

def srt_conv(subs):
    if subs:
        starttimes=[]
        endtimes=[]
        words=[]
        indexes=[]
        #"-->" in line
        subs=io.StringIO(subs)
        prevLineWasWords =False
        prevLineWasTime =False
        for line in subs:
            line=line.strip()
            if "-->" in line:
                starttimes.append(line[:8])
                endtimes.append(line[17:25])
                prevLineWasTime =True
            elif line.strip().isdigit() and not prevLineWasTime:
                indexes.append(int(line))
                prevLineWasTime=False
            else:
            #elif len(line) > 1:
                prevLineWasTime = False
                if prevLineWasWords:
                        words[len(words)-1] = words[len(words)-1] + " " + line
                else:
                        words.append(line)
                prevLineWasWords = True
                continue
            prevLineWasWords = False  
        
        #trim because sometimes a file will end without a last tine.
        trim_by=min([len(starttimes),len(endtimes),len(words)])
        starttimes=starttimes[0:trim_by]
        endtimes=endtimes[0:trim_by]
        words=words[0:trim_by]
        
        return starttimes, endtimes, words, indexes
    else:
        pass
    
def watch_output(clean_srt):
    subs=io.StringIO(clean_srt)
    print('=== is number, ~~~ is text, --- is time index. \n')
    prevLineWasTime=False
    for line in subs:
        if "-->" in line:
            prevLineWasTime=True
            print('----'+line)
        elif line.strip().isdigit() and not prevLineWasTime:
            prevLineWasTime=False
            print('====='+line)
        else:
        #elif len(line)>1:
            prevLineWasTime=False
            print('~~~~'+line)
    return

def s_time(string):
    try:
        a=datetime.strptime(string, '%H:%M:%S')
        return a
    except:
        pass
    

def make_df(imdb_id):
    srt1=fparse1(read_srt(imdb_id))
    if srt1:
        a,b,c,_=srt_conv(srt1)

        imdb_idi=[imdb_id]
        linfo=['StartTime','EndTime','Line']
        index=pd.MultiIndex.from_product([imdb_idi,linfo],names=['IDs','LData'])
        data_sub=pd.DataFrame(columns=index)


        s1 = pd.Series([s_time(item) for item in a])
        s2 = pd.Series([s_time(item) for item in b])
        s3 = pd.Series(c)
        data_sub.loc[:,(imdb_id,'StartTime')] =s1.values
        data_sub.loc[:,(imdb_id,'EndTime')] = s2.values
        data_sub.loc[:,(imdb_id,'Line')] = s3.values

        return data_sub
    else:
        pass
    
    
def make_tup(imdb_id):
    srt1=fparse1(read_srt(imdb_id))
    if srt1:
        a,b,c,_=srt_conv(srt1)

        s1 = [s_time(item) for item in a]
        s2 = [s_time(item) for item in b]
        s3 = c

        return (s1,s2,s3)
    else:
        pass    

def append_pd(df1,df):
    tes=pd.concat([df1,df], axis=1)
    tes=tes.fillna('')
    return tes

In [133]:
aid=[item.split('.')[0] for item in listdir(subs_dir) if 'tt' in item]

In [138]:
t=time.time()
full_data={}
for i in range(len(aid)):
    full_data[aid[i]]=make_tup(aid[i])
    if i%100==0:
        print('No. {} - adding {} - elapsed {} sec.'.format(i,aid[i],time.time()-t))
t1=time.time()
print('elapsed time {} seconds'.format(t1-t))

No. 0 - adding tt0002381 - elapsed 0.058014869689941406 sec.
No. 100 - adding tt0029583 - elapsed 2.318572759628296 sec.
No. 200 - adding tt0037522 - elapsed 6.224541425704956 sec.
No. 300 - adding tt0044081 - elapsed 10.13450312614441 sec.
No. 400 - adding tt0049949 - elapsed 13.515328407287598 sec.
No. 500 - adding tt0056193 - elapsed 17.217242002487183 sec.
No. 600 - adding tt0062190 - elapsed 20.68910241127014 sec.
No. 700 - adding tt0067328 - elapsed 24.059925317764282 sec.
No. 800 - adding tt0072431 - elapsed 27.283752918243408 sec.
No. 900 - adding tt0078736 - elapsed 31.405113220214844 sec.
No. 1000 - adding tt0082817 - elapsed 35.27142262458801 sec.
No. 1100 - adding tt0086837 - elapsed 39.01010704040527 sec.
No. 1200 - adding tt0090297 - elapsed 42.65796709060669 sec.
No. 1300 - adding tt0093223 - elapsed 46.41611313819885 sec.
No. 1400 - adding tt0096328 - elapsed 50.37458324432373 sec.
No. 1500 - adding tt0099557 - elapsed 54.393736600875854 sec.
No. 1600 - adding tt0103010